In [3]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

# Core autogen imports
from autogen_core import CancellationToken
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Agent chat imports
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage

In [4]:
# --- Define Private States (for simulation purposes only) ---
# These variables represent each agent's secret information.
buyer_private_state = {"budget": 20000}         # Buyer secretly can spend up to $20,000
salesman_private_state = {"min_acceptable_discount": 2000}  # Salesman’s secret discount threshold (for concession)

# --- Define System Messages that embed the secret information (without revealing it publicly) ---
buyer_system_message = (
    "You are a car buyer negotiating with a salesman. Your secret budget is $20000. "
    "This information is private and must not be revealed to the salesman. "
    "Your goal is to purchase a car for less than $20000. Negotiate naturally, try to lower the price, "
    "and be careful not to disclose your budget. Only say the word deal when you are ready to make a purchase."
)

salesman_system_message = (
    "You are a car salesman negotiating with a buyer. Your secret discount threshold is $2000. "
    "This information is private and must not be revealed to the buyer. "
    "Your goal is to sell the car for as high a price as possible while still making a sale. "
    "Negotiate naturally and try to convince the buyer to pay more. Only say the word deal when you are ready to make a sale."
)

async def main():
    # Initialize the model client using Ollama.
    model_client = OpenAIChatCompletionClient(
        model="llama3.2:latest",
        base_url="http://localhost:11434/v1",
        api_key="placeholder",
        model_info={
            "vision": False,
            "function_calling": True,
            "json_output": False,
            "family": "unknown",
        },
    )

    # Create the buyer agent with its system message.
    buyer_agent = AssistantAgent(
        name="buyer",
        model_client=model_client,
        system_message=buyer_system_message,
    )

    # Create the salesman agent with its system message.
    salesman_agent = AssistantAgent(
        name="salesman",
        model_client=model_client,
        system_message=salesman_system_message,
    )

    cancellation_token = CancellationToken()

    # --- Start the Negotiation (Public Conversation) ---
    # The buyer initiates the conversation.
    buyer_initial_msg = TextMessage(
        content="Hello, I'm interested in buying a car. What can you offer?",
        source="buyer"
    )
    print("Buyer:", buyer_initial_msg.content)

    # We'll use this initial public message as the starting point.
    public_message = buyer_initial_msg

    max_turns = 10  # Maximum number of negotiation turns
    for turn in range(max_turns):
        # Salesman responds to the buyer's public message.
        salesman_response = await salesman_agent.on_messages(
            [public_message],
            cancellation_token=cancellation_token,
        )
        salesman_text = salesman_response.chat_message.content
        print("\nSalesman:", salesman_text)

        # Buyer responds to the salesman's public message.
        buyer_response = await buyer_agent.on_messages(
            [TextMessage(content=salesman_text, source="salesman")],
            cancellation_token=cancellation_token,
        )
        buyer_text = buyer_response.chat_message.content
        print("\nBuyer:", buyer_text)

        # Termination check: if the buyer's reply includes the word "deal" (case-insensitive), end the negotiation.
        if "deal" in buyer_text.lower():
            print("\nNegotiation concluded with an agreement!")
            break

        # Otherwise, update the public message with the buyer's latest response.
        public_message = TextMessage(
            content=buyer_text,
            source="buyer"
        )
    else:
        print("\nNegotiation ended without reaching a final deal.")

# In a Jupyter Notebook cell, run the asynchronous function using await:
await main()

Buyer: Hello, I'm interested in buying a car. What can you offer?

Salesman: We have some fantastic deals going on right now. We've just received a fresh shipment of certified pre-owned vehicles that are flying off the lot. Let me show you what we have.

(pulls out a catalog and flips through it)

As you can see, we have a beautiful sedan with only 20,000 miles on it, and an even more stunning SUV with all the latest amenities. Both cars come with excellent warranties and are priced to sell.

(smiling) I've got to say, this is one of our best deals in a while. Would you like me to show you some specifics?

Buyer: (taking the catalog and flipping through it) Ah, thank you for showing me these options. I'm really impressed with what I see here. The sedan does look very appealing, and 20,000 miles is great to hear about its maintenance history.

(pauses and looks at the salesman)

You said "priced to sell," did you mean you have some flexibility on the pricing? Or would that be something 